# Level 5 Kaggle Reference Model
Author: **Guido Zuidhof** - [gzuidhof@lyft.com](mailto:gzuidhof@lyft.com)

#### D. Inference and postprocessing
4. Predicting our validation set.
5. Thresholding the probability map.
6. Performing a morphological closing operation to filter out tiny objects (presuming they are false positives)
7. Loading the ground truth
8. backprojecting our predicted boxes into world space

#### E. Visualizing the results (not included in this kernel)
x. Creating top down visualizations of the ground truth and predictions using the nuScenes SDK.  
x. (Optional) Creating a GIF of a scene.  

#### F. Evaluation
x. Computing mAP.

In [1]:
# Our code will generate data, visualization and model checkpoints, they will be persisted to disk in this folder
ARTIFACTS_FOLDER = "./artifacts"

In [2]:
!ls {ARTIFACTS_FOLDER}

bev_train_data		      unet_checkpoint_epoch_15.pth
bev_validation_data	      unet_checkpoint_epoch_1.pth
detection_boxes.npy	      unet_checkpoint_epoch_2.pth
detection_classes.npy	      unet_checkpoint_epoch_3.pth
detection_scores.npy	      unet_checkpoint_epoch_4.pth
pred_box3ds.npy		      unet_checkpoint_epoch_5.pth
unet_checkpoint_epoch_10.pth  unet_checkpoint_epoch_6.pth
unet_checkpoint_epoch_11.pth  unet_checkpoint_epoch_7.pth
unet_checkpoint_epoch_12.pth  unet_checkpoint_epoch_8.pth
unet_checkpoint_epoch_13.pth  unet_checkpoint_epoch_9.pth
unet_checkpoint_epoch_14.pth


In [3]:
import pdb
from datetime import datetime
from functools import partial
import glob
from multiprocessing import Pool

# Disable multiprocesing for numpy/opencv. We already multiprocess ourselves, this would mean every subprocess produces
# even more threads which would lead to a lot of context switching, slowing things down a lot.
import os
os.environ["OMP_NUM_THREADS"] = "1"

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm, tqdm_notebook
import scipy
import scipy.ndimage
import scipy.special
from scipy.spatial.transform import Rotation as R

from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix

In [4]:
base_dir = '../data/lyft/lyft_trainval/'

In [5]:
level5data = LyftDataset(data_path=base_dir, json_path=f'{base_dir}/v1.0-trainval', verbose=True)
os.makedirs(ARTIFACTS_FOLDER, exist_ok=True)

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 12.0 seconds.
Reverse indexing ...
Done reverse indexing in 2.3 seconds.


Preparing a dataframe of scenes, with one scene per row

In [6]:
# "bev" stands for birds eye view
classes = ["car", "motorcycle", "bus", "bicycle", "truck", "pedestrian", "other_vehicle", "animal", "emergency_vehicle"]
train_data_folder = os.path.join(ARTIFACTS_FOLDER, "bev_train_data")
validation_data_folder = os.path.join(ARTIFACTS_FOLDER, "./bev_validation_data")

lidar not able to load: lidar tokens
9cb04b1a4d476fd0782431764c7b55e91c6dbcbc6197c3dab3e044f13d058011

## C. Training a network to segment objects

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

class BEVImageDataset(torch.utils.data.Dataset):
    def __init__(self, input_filepaths, target_filepaths, map_filepaths=None):
        self.input_filepaths = input_filepaths
        self.target_filepaths = target_filepaths
        self.map_filepaths = map_filepaths
        
        if map_filepaths is not None:
            assert len(input_filepaths) == len(map_filepaths)
        
        assert len(input_filepaths) == len(target_filepaths)

    def __len__(self):
        return len(self.input_filepaths)

    def __getitem__(self, idx):
        input_filepath = self.input_filepaths[idx]
        target_filepath = self.target_filepaths[idx]
        
        sample_token = input_filepath.split("/")[-1].replace("_input.png","")
        
        im = cv2.imread(input_filepath, cv2.IMREAD_UNCHANGED)
        
        if self.map_filepaths:
            map_filepath = self.map_filepaths[idx]
            map_im = cv2.imread(map_filepath, cv2.IMREAD_UNCHANGED)
            im = np.concatenate((im, map_im), axis=2)
        
        target = cv2.imread(target_filepath, cv2.IMREAD_UNCHANGED)
        
        im = im.astype(np.float32)/255
        target = target.astype(np.int64)
        
        im = torch.from_numpy(im.transpose(2,0,1))
        target = torch.from_numpy(target)
        
        return im, target, sample_token

In [8]:
# This implementation was copied from https://github.com/jvanvugt/pytorch-unet, it is MIT licensed.

class UNet(nn.Module):
    def __init__(
        self,
        in_channels=1,
        n_classes=2,
        depth=5,
        wf=6,
        padding=False,
        batch_norm=False,
        up_mode='upconv',
    ):
        """
        Implementation of
        U-Net: Convolutional Networks for Biomedical Image Segmentation
        (Ronneberger et al., 2015)
        https://arxiv.org/abs/1505.04597
        Using the default arguments will yield the exact version used
        in the original paper
        Args:
            in_channels (int): number of input channels
            n_classes (int): number of output channels
            depth (int): depth of the network
            wf (int): number of filters in the first layer is 2**wf
            padding (bool): if True, apply padding such that the input shape
                            is the same as the output.
                            This may introduce artifacts
            batch_norm (bool): Use BatchNorm after layers with an
                               activation function
            up_mode (str): one of 'upconv' or 'upsample'.
                           'upconv' will use transposed convolutions for
                           learned upsampling.
                           'upsample' will use bilinear upsampling.
        """
        super(UNet, self).__init__()
        assert up_mode in ('upconv', 'upsample')
        self.padding = padding
        self.depth = depth
        prev_channels = in_channels
        self.down_path = nn.ModuleList()
        for i in range(depth):
            self.down_path.append(
                UNetConvBlock(prev_channels, 2 ** (wf + i), padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.up_path = nn.ModuleList()
        for i in reversed(range(depth - 1)):
            self.up_path.append(
                UNetUpBlock(prev_channels, 2 ** (wf + i), up_mode, padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.last = nn.Conv2d(prev_channels, n_classes, kernel_size=1)

    def forward(self, x):
        blocks = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            if i != len(self.down_path) - 1:
                blocks.append(x)
                x = F.max_pool2d(x, 2)

        for i, up in enumerate(self.up_path):
            x = up(x, blocks[-i - 1])

        return self.last(x)


class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, padding, batch_norm):
        super(UNetConvBlock, self).__init__()
        block = []

        block.append(nn.Conv2d(in_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        block.append(nn.Conv2d(out_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        self.block = nn.Sequential(*block)

    def forward(self, x):
        out = self.block(x)
        return out


class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, up_mode, padding, batch_norm):
        super(UNetUpBlock, self).__init__()
        if up_mode == 'upconv':
            self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2, stride=2)
        elif up_mode == 'upsample':
            self.up = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_size, out_size, kernel_size=1),
            )

        self.conv_block = UNetConvBlock(in_size, out_size, padding, batch_norm)

    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[
            :, :, diff_y : (diff_y + target_size[0]), diff_x : (diff_x + target_size[1])
        ]

    def forward(self, x, bridge):
        up = self.up(x)
        crop1 = self.center_crop(bridge, up.shape[2:])
        out = torch.cat([up, crop1], 1)
        out = self.conv_block(out)

        return out

We train a U-net fully convolutional neural network, we create a network that is less deep and with only half the amount of filters compared to the original U-net paper implementation. We do this to keep training and inference time low.

In [9]:
def get_unet_model(in_channels=3, num_output_classes=2):
    model = UNet(in_channels=in_channels, n_classes=num_output_classes, wf=5, depth=4, padding=True, up_mode='upsample')
    # Optional, for multi GPU training and inference
    model = nn.DataParallel(model)
    return model


In [10]:
def visualize_predictions(input_image, prediction, target, n_images=2, apply_softmax=True):
    """
    Takes as input 3 PyTorch tensors, plots the input image, predictions and targets.
    """
    # Only select the first n images
    prediction = prediction[:n_images]
    target = target[:n_images]
    input_image = input_image[:n_images]

    prediction = prediction.detach().cpu().numpy()
    if apply_softmax:
        prediction = scipy.special.softmax(prediction, axis=1)
    class_one_preds = np.hstack(1-prediction[:,0])

    target = np.hstack(target.detach().cpu().numpy())

    class_rgb = np.repeat(class_one_preds[..., None], 3, axis=2)
    class_rgb[...,2] = 0
    class_rgb[...,1] = target

    
    input_im = np.hstack(input_image.cpu().numpy().transpose(0,2,3,1))
    
    if input_im.shape[2] == 3:
        input_im_grayscale = np.repeat(input_im.mean(axis=2)[..., None], 3, axis=2)
        overlayed_im = (input_im_grayscale*0.6 + class_rgb*0.7).clip(0,1)
    else:
        input_map = input_im[...,3:]
        overlayed_im = (input_map*0.6 + class_rgb*0.7).clip(0,1)

    thresholded_pred = np.repeat(class_one_preds[..., None] > 0.5, 3, axis=2)

    fig = plt.figure(figsize=(12,26))
    plot_im = np.vstack([class_rgb, input_im[...,:3], overlayed_im, thresholded_pred]).clip(0,1).astype(np.float32)
    plt.imshow(plot_im)
    plt.axis("off")
    plt.show()

In [11]:
# We weigh the loss for the 0 class lower to account for (some of) the big class imbalance.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_weights = torch.from_numpy(np.array([0.2] + [1.0]*len(classes), dtype=np.float32))
class_weights = class_weights.to(device)

In [12]:
class_weights, classes, class_weights.shape

(tensor([0.2000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000], device='cuda:0'),
 ['car',
  'motorcycle',
  'bus',
  'bicycle',
  'truck',
  'pedestrian',
  'other_vehicle',
  'animal',
  'emergency_vehicle'],
 torch.Size([10]))

total 10 classes: background + 9 categories, 0.2 weight to bg, rest 1

### training

In [ ]:
batch_size = 8
epochs = 15 # Note: We may be able to train for longer and expect better results, the reason this number is low is to keep the runtime short.

model = get_unet_model(num_output_classes=len(classes)+1)
model = model.to(device)

optim = torch.optim.Adam(model.parameters(), lr=1e-3)
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, num_workers=os.cpu_count()*2)

all_losses = []

for epoch in range(1, epochs+1):
    print("Epoch", epoch)
    
    epoch_losses = []
    progress_bar = tqdm_notebook(dataloader)
    
    for ii, (X, target, sample_ids) in enumerate(progress_bar):
        X = X.to(device)  # [N, 3, H, W]
        target = target.to(device)  # [N, H, W] with class indices (0, 1)
        prediction = model(X)  # [N, 2, H, W]
        loss = F.cross_entropy(prediction, target, weight=class_weights)

        optim.zero_grad()
        loss.backward()
        optim.step()
        
        epoch_losses.append(loss.detach().cpu().numpy())

        if ii == 0:
            visualize_predictions(X, prediction, target)
    
    print("Loss:", np.mean(epoch_losses))
    all_losses.extend(epoch_losses)
    
    checkpoint_filename = "unet_checkpoint_epoch_{}.pth".format(epoch)
    checkpoint_filepath = os.path.join(ARTIFACTS_FOLDER, checkpoint_filename)
    torch.save(model.state_dict(), checkpoint_filepath)
    
plt.figure(figsize=(12,12))
plt.plot(all_losses, alpha=0.75)
plt.show()

#### You can interpret the above visualizations as follows:  
There are four different visualizations stacked on top of eachother:
1. The top images have two color channels: red for predictions, green for targets. Note that red+green=yellow. In other words:  
> **Black**: True Negative  
**Green**: False Negative  
**Yellow**: True Positive  
**Red**: False Positive 
2. The input image
3. The input image (or semantic input map, not in this kernel) blended together with targets+predictions
4. The predictions thresholded at 0.5 probability.

### val set predictions

In [13]:
input_filepaths = sorted(glob.glob(os.path.join(validation_data_folder, "*_input.png")))
target_filepaths = sorted(glob.glob(os.path.join(validation_data_folder, "*_target.png")))

batch_size=16
validation_dataset = BEVImageDataset(input_filepaths, target_filepaths)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size, shuffle=False, num_workers=os.cpu_count())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_unet_model(num_output_classes=1+len(classes))
model = model.to(device)

epoch_to_load=15
checkpoint_filename = "unet_checkpoint_epoch_{}.pth".format(epoch_to_load)
checkpoint_filepath = os.path.join(ARTIFACTS_FOLDER, checkpoint_filename)
model.load_state_dict(torch.load(checkpoint_filepath))

<All keys matched successfully>

In [14]:
progress_bar = tqdm_notebook(validation_dataloader)

targets = np.zeros((len(target_filepaths), 336, 336), dtype=np.uint8)

# We quantize to uint8 here to conserve memory. We're allocating >20GB of memory otherwise.
predictions = np.zeros((len(target_filepaths), 1+len(classes), 336, 336), dtype=np.uint8)

sample_tokens = []
all_losses = []

with torch.no_grad():
    model.eval()
    for ii, (X, target, batch_sample_tokens) in enumerate(progress_bar):

        offset = ii*batch_size
        targets[offset:offset+batch_size] = target.numpy()
        sample_tokens.extend(batch_sample_tokens)
#         continue
        X = X.to(device)  # [N, 1, H, W]
        target = target.to(device)  # [N, H, W] with class indices (0, 1)
        prediction = model(X)  # [N, 2, H, W]
        loss = F.cross_entropy(prediction, target, weight=class_weights)
        all_losses.append(loss.detach().cpu().numpy())
        
        prediction = F.softmax(prediction, dim=1)
        
        prediction_cpu = prediction.cpu().numpy()
        predictions[offset:offset+batch_size] = np.round(prediction_cpu*255).astype(np.uint8)
        
        # Visualize the first prediction
        if ii == 0:
            pass
#             visualize_predictions(X, prediction, target, apply_softmax=False)
            
print("Mean loss:", np.mean(all_losses))


Mean loss: nan


/home/ags/miniconda3/envs/ML/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ags/miniconda3/envs/ML/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [46]:
np.round(prediction_cpu * 255)[0][:, 0, 0]

array([253.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
      dtype=float32)

In [47]:
# del model, class_weights, validation_dataloader, validation_dataset

In [48]:
predictions.shape, prediction.shape

((5040, 10, 336, 336), torch.Size([16, 10, 336, 336]))

In [49]:
predictions[0][:, 0, 0], len(classes)

(array([252,   2,   0,   0,   0,   0,   0,   0,   0,   0], dtype=uint8), 9)

In [59]:
# Get probabilities for non-background
predictions_non_class0 = 255 - predictions[:,0]

In [51]:
predictions_non_class0.shape

(5040, 336, 336)

In [53]:
# Arbitrary threshold in our system to create a binary image to fit boxes around.
background_threshold = 255//2

#for i in range(3):
#    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))
#    axes[0].imshow(predictions_non_class0[i])
#    axes[0].set_title("predictions")
#    axes[1].imshow(predictions_non_class0[i] > background_threshold)
#    axes[1].set_title("thresholded predictions")
#    axes[2].imshow((targets[i] > 0).astype(np.uint8), interpolation="nearest")
#    axes[2].set_title("targets")
#    fig.tight_layout()
#    fig.show()

'\nfor i in range(3):\n    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 6))\n    axes[0].imshow(predictions_non_class0[i])\n    axes[0].set_title("predictions")\n    axes[1].imshow(predictions_non_class0[i] > background_threshold)\n    axes[1].set_title("thresholded predictions")\n    axes[2].imshow((targets[i] > 0).astype(np.uint8), interpolation="nearest")\n    axes[2].set_title("targets")\n    fig.tight_layout()\n    fig.show()\n'

In [54]:
# We perform an opening morphological operation to filter tiny detections
# Note that this may be problematic for classes that are inherently small (e.g. pedestrians)..
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
predictions_opened = np.zeros((predictions_non_class0.shape), dtype=np.uint8)

for i, p in enumerate(tqdm(predictions_non_class0)):
    thresholded_p = (p > background_threshold).astype(np.uint8)
    predictions_opened[i] = cv2.morphologyEx(thresholded_p, cv2.MORPH_OPEN, kernel)

#plt.figure(figsize=(12,12))
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
#axes[0].imshow(predictions_non_class0[0] > 255//2)
#axes[0].set_title("thresholded prediction")
#axes[1].imshow(predictions_opened[0])
#axes[1].set_title("opened thresholded prediction")
#fig.show()

100%|██████████| 5040/5040 [00:00<00:00, 10966.43it/s]


In [55]:
# Sanity check: let's count the amount of connected components in an image
# labels, n = scipy.ndimage.label(predictions_opened[0])
# plt.imshow(labels, cmap="tab20b")
# plt.xlabel("N predictions: {}".format(n))
# plt.show()

The above image looks pretty well separated, some boxes seem to be wrongly merged together and may be problematic. Let's continue.
For each scene we fit boxes to the segmentations. For each box and each class we write it's probability in the center pixel.

In [53]:
# del labels

In [64]:
np.unique(predictions_opened[0]), np.unique(predictions_non_class0[0])

(array([0, 1], dtype=uint8),
 array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171,

In [61]:
predictions_opened.shape, predictions_non_class0.shape, predictions.shape

((5040, 336, 336), (5040, 336, 336), (5040, 10, 336, 336))

In [65]:
detection_boxes = []
detection_scores = []
detection_classes = []

for i in tqdm_notebook(range(len(predictions))):
    prediction_opened = predictions_opened[i]
    probability_non_class0 = predictions_non_class0[i]
    class_probability = predictions[i]

    sample_boxes = []
    sample_detection_scores = []
    sample_detection_classes = []
    
    contours, hierarchy = cv2.findContours(prediction_opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    
    for cnt in contours:
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        
        # Let's take the center pixel value as the confidence value
        box_center_index = np.int0(np.mean(box, axis=0))
        
        for class_index in range(len(classes)):
            box_center_value = class_probability[class_index+1, box_center_index[1], box_center_index[0]] # notice the [1, 0] ?
            
            # Let's remove candidates with very low probability
            if box_center_value < 0.01:
                continue
            
            box_center_class = classes[class_index]

            box_detection_score = box_center_value
            sample_detection_classes.append(box_center_class)
            sample_detection_scores.append(box_detection_score)
            sample_boxes.append(box)
        
    
    detection_boxes.append(np.array(sample_boxes))
    detection_scores.append(sample_detection_scores)
    detection_classes.append(sample_detection_classes)
    
print("Total amount of boxes:", np.sum([len(x) for x in detection_boxes]))
    
# # Visualize the boxes in the first sample
# t = np.zeros_like(predictions_opened[0])
# for sample_boxes in detection_boxes[0]:
#     box_pix = np.int0(sample_boxes)
#     cv2.drawContours(t,[box_pix],0,(255),2)
# plt.imshow(t)
# plt.show()

# # Visualize their probabilities
# plt.hist(detection_scores[0], bins=20)
# plt.xlabel("Detection Score")
# plt.ylabel("Count")
# plt.show()


Total amount of boxes: 148861


Let's load the ground truth for the validation set.

In [ ]:
# !export DYLD_FALLBACK_LIBRARY_PATH=$(HOME)/lib:/usr/local/lib:/lib:/usr/lib

In [15]:
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision

def load_groundtruth_boxes(nuscenes, sample_tokens):
    gt_box3ds = []

    # Load annotations and filter predictions and annotations.
    for sample_token in tqdm_notebook(sample_tokens):

        sample = nuscenes.get('sample', sample_token)
        sample_annotation_tokens = sample['anns']

        sample_lidar_token = sample["data"]["LIDAR_TOP"]
        lidar_data = level5data.get("sample_data", sample_lidar_token)
        ego_pose = level5data.get("ego_pose", lidar_data["ego_pose_token"])
        ego_translation = np.array(ego_pose['translation'])
        
        for sample_annotation_token in sample_annotation_tokens:
            sample_annotation = nuscenes.get('sample_annotation', sample_annotation_token)
            sample_annotation_translation = sample_annotation['translation']
            
            class_name = sample_annotation['category_name']
            
            box3d = Box3D(
                sample_token=sample_token,
                translation=sample_annotation_translation,
                size=sample_annotation['size'],
                rotation=sample_annotation['rotation'],
                name=class_name
            )
            gt_box3ds.append(box3d)
            
    return gt_box3ds

gt_box3ds = load_groundtruth_boxes(level5data, sample_tokens)

Next we take our predicted boxes, transform them back into world space and make them 3D.

In [16]:
# np.save(os.path.join(ARTIFACTS_FOLDER, 'gt_box3ds.npy'), gt_box3ds)
# np.save(os.path.join(ARTIFACTS_FOLDER, 'detection_boxes.npy'), detection_boxes)
# np.save(os.path.join(ARTIFACTS_FOLDER, 'detection_scores.npy'), detection_scores)
# np.save(os.path.join(ARTIFACTS_FOLDER, 'detection_classes.npy'), detection_classes)

In [17]:
def create_transformation_matrix_to_voxel_space(shape, voxel_size, offset):
    """
    Constructs a transformation matrix given an output voxel shape such that (0,0,0) ends up in the center.
    Voxel_size defines how large every voxel is in world coordinate, (1,1,1) would be the same as Minecraft voxels.
    
    An offset per axis in world coordinates (metric) can be provided, this is useful for Z (up-down) in lidar points.
    """
    
    shape, voxel_size, offset = np.array(shape), np.array(voxel_size), np.array(offset)
    
    tm = np.eye(4, dtype=np.float32)
    translation = shape/2 + offset/voxel_size
    
    tm = tm * np.array(np.hstack((1/voxel_size, [1])))
    tm[:3, 3] = np.transpose(translation)
    return tm

def transform_points(points, transf_matrix):
    """
    Transform (3,N) or (4,N) points using transformation matrix.
    """
    if points.shape[0] not in [3,4]:
        raise Exception("Points input should be (3,N) or (4,N) shape, received {}".format(points.shape))
    return transf_matrix.dot(np.vstack((points[:3, :], np.ones(points.shape[1]))))[:3, :]


In [18]:
# Some hyperparameters we'll need to define for the system
voxel_size = (0.4, 0.4, 1.5)
z_offset = -2.0
bev_shape = (336, 336, 3)

# We scale down each box so they are more separated when projected into our coarse voxel space.
box_scale = 0.8

In [19]:
gt_box3ds = np.load(os.path.join(ARTIFACTS_FOLDER, 'gt_box3ds.npy'))
detection_boxes = np.load(os.path.join(ARTIFACTS_FOLDER, 'detection_boxes.npy'))
detection_scores = np.load(os.path.join(ARTIFACTS_FOLDER, 'detection_scores.npy'))
detection_classes = np.load(os.path.join(ARTIFACTS_FOLDER, 'detection_classes.npy'))

In [22]:
len(gt_box3ds), len(detection_boxes), len(detection_scores), len(detection_classes)

(143414, 5040, 5040, 5040)

In [24]:
detection_boxes[0][0]

array([[204.00003, 266.00003],
       [197.50003, 259.50003],
       [202.00003, 255.00003],
       [208.50003, 261.50003]], dtype=float32)

In [25]:
pred_box3ds = []

# This could use some refactoring..
for (sample_token, sample_boxes, sample_detection_scores, sample_detection_class) in tqdm_notebook(zip(sample_tokens, detection_boxes, detection_scores, detection_classes), total=len(sample_tokens)):
    sample_boxes = sample_boxes.reshape(-1, 2) # (N, 4, 2) -> (N*4, 2)
    sample_boxes = sample_boxes.transpose(1,0) # (N*4, 2) -> (2, N*4)

    # Add Z dimension
    sample_boxes = np.vstack((sample_boxes, np.zeros(sample_boxes.shape[1]),)) # (2, N*4) -> (3, N*4)

    sample = level5data.get("sample", sample_token)
    sample_lidar_token = sample["data"]["LIDAR_TOP"]
    lidar_data = level5data.get("sample_data", sample_lidar_token)
    lidar_filepath = level5data.get_sample_data_path(sample_lidar_token)
    ego_pose = level5data.get("ego_pose", lidar_data["ego_pose_token"])
    ego_translation = np.array(ego_pose['translation'])

    global_from_car = transform_matrix(ego_pose['translation'],
                                       Quaternion(ego_pose['rotation']), inverse=False)

    car_from_voxel = np.linalg.inv(create_transformation_matrix_to_voxel_space(bev_shape, voxel_size, (0, 0, z_offset)))


    global_from_voxel = np.dot(global_from_car, car_from_voxel)
    sample_boxes = transform_points(sample_boxes, global_from_voxel)

    # We don't know at where the boxes are in the scene on the z-axis (up-down), let's assume all of them are at
    # the same height as the ego vehicle.
    sample_boxes[2,:] = ego_pose["translation"][2]


    # (3, N*4) -> (N, 4, 3)
    sample_boxes = sample_boxes.transpose(1,0).reshape(-1, 4, 3)


    # We don't know the height of our boxes, let's assume every object is the same height.
    box_height = 1.75

    # Note: Each of these boxes describes the ground corners of a 3D box.
    # To get the center of the box in 3D, we'll have to add half the height to it.
    sample_boxes_centers = sample_boxes.mean(axis=1)
    sample_boxes_centers[:,2] += box_height/2

    # Width and height is arbitrary - we don't know what way the vehicles are pointing from our prediction segmentation
    # It doesn't matter for evaluation, so no need to worry about that here.
    # Note: We scaled our targets to be 0.8 the actual size, we need to adjust for that
    sample_lengths = np.linalg.norm(sample_boxes[:,0,:] - sample_boxes[:,1,:], axis=1) * 1/box_scale
    sample_widths = np.linalg.norm(sample_boxes[:,1,:] - sample_boxes[:,2,:], axis=1) * 1/box_scale
    
    sample_boxes_dimensions = np.zeros_like(sample_boxes_centers) 
    sample_boxes_dimensions[:,0] = sample_widths
    sample_boxes_dimensions[:,1] = sample_lengths
    sample_boxes_dimensions[:,2] = box_height

    for i in range(len(sample_boxes)):
        translation = sample_boxes_centers[i]
        size = sample_boxes_dimensions[i]
        class_name = sample_detection_class[i]
        ego_distance = float(np.linalg.norm(ego_translation - translation))
    
        
        # Determine the rotation of the box
        v = (sample_boxes[i,0] - sample_boxes[i,1])
        v /= np.linalg.norm(v)
        r = R.from_dcm([
            [v[0], -v[1], 0],
            [v[1],  v[0], 0],
            [   0,     0, 1],
        ])
        quat = r.as_quat()
        # XYZW -> WXYZ order of elements
        quat = quat[[3,0,1,2]]
        
        detection_score = float(sample_detection_scores[i])

        
        box3d = Box3D(
            sample_token=sample_token,
            translation=list(translation),
            size=list(size),
            rotation=list(quat),
            name=class_name,
            score=detection_score
        )
        pred_box3ds.append(box3d)


In [26]:
# np.save(os.path.join(ARTIFACTS_FOLDER, 'pred_box3ds.npy'), pred_box3ds)

In [ ]:
gt_box3ds = np.load(os.path.join(ARTIFACTS_FOLDER, 'gt_box3ds.npy'))
detection_boxes = np.load(os.path.join(ARTIFACTS_FOLDER, 'detection_boxes.npy'))
detection_scores = np.load(os.path.join(ARTIFACTS_FOLDER, 'detection_scores.npy'))
detection_classes = np.load(os.path.join(ARTIFACTS_FOLDER, 'detection_classes.npy'))
pred_box3ds = np.load(os.path.join(ARTIFACTS_FOLDER, 'pred_box3ds.npy'))

In [30]:
len(pred_box3ds), len(gt_box3ds) 

(148861, 143414)

In [31]:
pred_box3ds[0]

{'sample_token': '00336df4f44fa605a6ef6a08e8b312564d13f385228abe9719af6daa543033b2', 'translation': [2612.483725582335, 768.8829575895404, -18.772330722393747], 'size': [3.1818760303387217, 4.596178485914876, 1.75], 'rotation': [0.9917494534411332, 0.0, 0.0, 0.1281913475988669], 'name': 'car', 'volume': 25.59282277210936, 'score': 211.0}

In [32]:
gt_box3ds[0]

{'sample_token': '00336df4f44fa605a6ef6a08e8b312564d13f385228abe9719af6daa543033b2', 'translation': [2588.0412154737305, 744.1343975981417, -18.810591968737043], 'size': [1.653, 4.332, 1.444], 'rotation': [-0.9628021828085169, 0, 0, 0.27020724782869027], 'name': 'car', 'volume': 10.340189423999998, 'score': -1}

In [ ]:
gt_box3ds = [box.serialize() for box in gt_box3ds]
pred_box3ds = [box.serialize() for box in pred_box3ds]

In [48]:
from lyft_dataset_sdk.eval.detection import mAP_evaluation
iou_threshold = 0.5
average_precisions = mAP_evaluation.get_average_precisions(gt_box3ds, pred_box3ds, classes, iou_threshold)

/media/ags/DATA/CODE/kaggle/lyft-3d-object-detection/notebooks/lyft_dataset_sdk/eval/detection/mAP_evaluation.py:312: RuntimeWarning: invalid value encountered in true_divide
  recalls = tp / float(num_gts)
/media/ags/DATA/CODE/kaggle/lyft-3d-object-detection/notebooks/lyft_dataset_sdk/eval/detection/mAP_evaluation.py:314: RuntimeWarning: invalid value encountered in greater_equal
  assert np.all(0 <= recalls) & np.all(recalls <= 1)
/media/ags/DATA/CODE/kaggle/lyft-3d-object-detection/notebooks/lyft_dataset_sdk/eval/detection/mAP_evaluation.py:314: RuntimeWarning: invalid value encountered in less_equal
  assert np.all(0 <= recalls) & np.all(recalls <= 1)


AssertionError: 

In [61]:
gt_by_class_name = mAP_evaluation.group_by_key(gt_box3ds, "name")
pred_by_class_name = mAP_evaluation.group_by_key(pred_box3ds, "name")

In [62]:
for key in gt_by_class_name.keys():
    print(key, len(gt_by_class_name[key]))

car 118839
bicycle 5503
other_vehicle 7009
bus 2044
truck 4002
motorcycle 159
pedestrian 5824
animal 34


In [63]:
gt_by_class_name.keys()

dict_keys(['car', 'bicycle', 'other_vehicle', 'bus', 'truck', 'motorcycle', 'pedestrian', 'animal'])

In [64]:
for key in pred_by_class_name.keys():
    print(key, len(pred_by_class_name[key]))

car 107244
bicycle 8255
pedestrian 8058
bus 6178
truck 5945
other_vehicle 10603
motorcycle 2006
emergency_vehicle 543
animal 29


In [66]:
average_precisions = np.zeros(len(classes))

for class_id, class_name in enumerate(classes):
    if class_name in pred_by_class_name and class_name in gt_by_class_name:
        recalls, precisions, average_precision = recall_precision(
            gt_by_class_name[class_name], pred_by_class_name[class_name], iou_threshold
        )
        average_precisions[class_id] = average_precision


In [69]:
average_precisions#.mean()

array([2.63153498e-01, 0.00000000e+00, 2.33167296e-04, 5.57413055e-05,
       3.26573451e-05, 8.79200888e-05, 5.69355671e-04, 0.00000000e+00,
       0.00000000e+00])

> ⚠️ Note: This kernel is a work in progress

I didn't want to hold off on releasing this kernel as I think it will help with getting started in this competition as it is :). 

At this point we have `pred_box3ds` and `gt_box3ds`, they are the predictions and the targets on the validation set.
Next steps: 
* Compute mAP on the validation set using the evaluation script provided in the SDK
* Run inference on the test set.
* Making a submission.



### Model limitations
- The model performs very poorly on uncommon classes.
- The boxes are imprecise: the input has a very low resolution (one pixel is 40x40cm in the real world!), and we arbitrarily threshold the predictions and fit boxes around these boxes. As we evaluate with IoUs between 0.4 and 0.75, we can expect that to hurt the score.
- The model is barely converged - we could train for longer.
- We only use LIDAR data, and we only use one lidar sweep.
- We compress the height dimension into only 3 channels. We assume every object is 1.75 meters tall and is at the same height of the ego vehicle, which is surely a wrong assumption.

In [ ]:
import shutil
shutil.rmtree(train_data_folder)
shutil.rmtree(validation_data_folder)